In [45]:
import numpy as np
import rasterio

def normalize_bands(src):
    bands_normalized = []
    for i in range(1, src.count + 1):
        band = src.read(i)
        p1, p99 = np.percentile(band, [1, 99])
        band_normalized = np.clip((band - p1) / (p99 - p1), 0, 1)
        bands_normalized.append(band_normalized)
    return bands_normalized

def process_and_save_normalized_image(input_path, output_path):
    with rasterio.open(input_path, 'r') as src:
        # Read and normalize all bands first
        bands_normalized = normalize_bands(src)
        
        # Prepare new metadata for the output file
        new_meta = src.meta.copy()
        new_meta.update(dtype=rasterio.float32)

        # Write the normalized data to a new output file
        with rasterio.open(output_path, 'w', **new_meta) as dst:
            for i, band in enumerate(bands_normalized, start=1):
                dst.write(band, i)

# Define the paths
input_path = "/workspaces/Satelite/data/satellite/aargau_parcel_53.tif"
output_path = "/workspaces/Satelite/data/satellite/aargau_parcel_53_normalized.tif"

# Process and save the normalized image
process_and_save_normalized_image(input_path, output_path)
